In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn import metrics

target_path = 'val_prediction.csv.03-04-2018_16:07:37'
dtypes = pickle.load(open('output_dtypes.pickle','rb'))

data = pd.read_csv(target_path, dtype=dtypes,parse_dates=["click_time"])

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6303449 entries, 0 to 6303448
Data columns (total 23 columns):
click_time              datetime64[ns]
ip                      uint64
app                     uint16
device                  uint16
os                      uint16
channel                 uint16
is_attributed           uint8
hour                    uint8
day                     uint8
ip_day_hourcount        uint16
ip_appcount             uint16
ip_channelcount         uint16
ip_channel_appcount     uint16
ip_app_oscount          uint16
ipcount                 uint16
ip_hour_channelcount    uint16
ip_hour_oscount         uint16
ip_hour_appcount        uint16
channel_appcount        uint16
channel_oscount         uint16
channel_app_oscount     uint16
app_oscount             uint16
predict                 float64
dtypes: datetime64[ns](1), float64(1), uint16(17), uint64(1), uint8(3)
memory usage: 366.7 MB


In [2]:
def calculate_auc(val):
    #if getattr(val['predict'], "values", None) is None:
    #    return 0
    fpr, tpr, thresholds = metrics.roc_curve(val['is_attributed'].values,
        val['predict'].values)
    auc = metrics.auc(fpr, tpr)
    return auc

print('overall auc:', calculate_auc(data))

overall auc: 0.968729523171


In [5]:
def top_n_values_of_dimensions(data, dimensions,count_col):
    d1 = data[dimensions + [count_col]+['is_attributed','predict']].\
        groupby(by=dimensions)[[count_col]].count().\
        sort_values(count_col, ascending=False).rename(columns={count_col:'_'.join(dimensions+['count'])})
    d1 = d1.query('_'.join(dimensions+['count']) + ' > 50')
    d1['_'.join(dimensions+['ratio'])] = d1['_'.join(dimensions+['count'])] / len(data)
        #n_chans = training_hist_attribution[group_by_cols + [counting_col]] \
        #    .groupby(by=group_by_cols)[[counting_col]] \
        #    .count().reset_index().rename(columns={counting_col: feature_name_added1})
    d1=d1.reset_index()
    d1['_'.join(dimensions)] = pd.Series(['_'. \
        join([str(row[dimension]) for dimension in dimensions]) for index, row in d1.iterrows()])
    data['_'.join(dimensions)] = pd.Series(['_'. \
        join([str(row[dimension]) for dimension in dimensions]) for index, row in data.iterrows()])
    
    data = data.set_index('_'.join(dimensions))
    #data = data.set_index(dimensions)
    
    
    d1['_'.join(dimensions + ['auc'])] = pd.Series([calculate_auc(data.loc['_'. \
        join([str(row[dimension]) for dimension in dimensions])]) for index, row in d1.iterrows()])
    
    #print('dataloc:',data.loc['178_1'])
    
    #print([row for index, row in d1.reset_index().iterrows()])   

    #print(data.loc[[[row[dimension]] for dimension in dimensions] for index, row in d1.reset_index().iterrows()])   
    #print([calculate_auc(data.loc[row[dimensions]]) for index, row in d1.iterrows()])  
    
    #for index, row in d1.reset_index().iterrows():
    #    l = [str(row[dimension]) for dimension in dimensions]
    #    print("l:%s:%s" % (index,'_'.join(l)))
    #    #print("data loc:", data.loc['_'.join(l)]['is_attributed'].describe())
    #    print("data loc auc: %s" % calculate_auc(data.loc['_'.join(l)]))

            

    return d1

import gc
gc.collect()
print(top_n_values_of_dimensions(data, ['channel'],'app'))

/Users/youku/software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


KeyError: 'the label [107.0] is not in the [index]'